In [ ]:
import argparse
import os
import sys
import time
from PIL import Image
import tensorflow as tf
import numpy as np
from scipy import misc

from model import FCN8s, PSPNet50, ENet, ICNet

save_dir = './output/'
model_path = {'pspnet': '/cvlabdata1/home/lis/pretrained/pspnet50.npy',
              'fcn': './model/fcn.npy',
              'enet': './model/cityscapes/enet.ckpt',
              'icnet': './model/cityscapes/icnet.npy'}



In [ ]:
from matplotlib import pyplot as plt

In [ ]:
psp_net = PSPNet50()

In [ ]:

# Init tf Session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.log_device_placement = True

sess = tf.Session(config=config)
init = tf.global_variables_initializer()

sess.run(init)

In [ ]:
# load weights
# set your weight path here
psp_net.load('/cvlabdata1/home/lis/pretrained/pspnet50.npy', sess)

In [ ]:
def exec_net(net, img_path):
	psp_net.read_input(img_path)
	result = sess.run([net.pred, psp_net.out_pre_softmax], feed_dict={net.x: net.img})
	return result

results = exec_net(psp_net, './input/outdoor_1.png')

In [ ]:
print(results[0].shape)
plt.imshow(results[0][0].astype(np.uint8))

In [ ]:
print(results[1].shape)
# (batch, H, W, sem class)
# show pre-softmax logits for class 0
plt.imshow(results[1][0][:, :, 0])

In [ ]:
# show pre-softmax logits for class 1
plt.imshow(results[1][0][:, :, 1])

In [ ]:
def test_class_ignore(logits_pre_softmax):
	
	result_with_all_classes = np.argmax(logits_pre_softmax, axis=3)
	
	logits_classes_removed = logits_pre_softmax.copy()
	logits_classes_removed[:, :, :, :75] = 0
	#logits_classes_removed[:, :, :, [3,5,7,9]] = 0
	
	result_with_selected_classes = np.argmax(logits_classes_removed, axis=3)
	
	plt.imshow(result_with_all_classes[0])
	plt.figure()
	plt.imshow(result_with_selected_classes[0])

test_class_ignore(results[1])